In [8]:
import os
import requests
from bs4 import BeautifulSoup, SoupStrainer
from urllib.parse import urljoin, urlparse

In [45]:
def is_valid(url):
    """Check if the URL is valid and not a parent directory link."""
    parsed = urlparse(url)
    return bool(parsed.netloc) and bool(parsed.scheme) and not url.endswith('../')

def get_all_files(url):
    """Retrieve all file URLs in the given directory URL, adhering to specific parsing rules."""
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    pre_content = soup.find('pre')

    # print(pre_content)
    if not pre_content:
        return []

    links = []
    start_collecting = False  # Flag to start collecting links after the back.gif
    skip_next_tag = False
    # Iterate through each element inside the <pre> tag
    for element in pre_content.contents:
        if element.name == 'img' and element.get('src') == '/icons/back.gif':
            start_collecting = True  # Start collecting links after this point
            skip_next_tag = True
            continue  # Skip the next link, which is the parent directory link
        if start_collecting:
            if element.name == 'hr':
                break  # Stop collecting after the <hr> tag
            if element.name == 'a':
                if skip_next_tag : 
                    skip_next_tag = False
                    continue
                href = element.get('href')
                href = url + "/" + href
                if is_valid(href):
                    links.append(href)

    return links

def download_files(url, download_path):
    """Download files from the given URL into the specified local directory."""
    files = get_all_files(url)
    for file in files:
        # If it's a directory, recurse into it
        if file.endswith('/'):
            next_path = os.path.join(download_path, file.strip('/').split('/')[-1])
            os.makedirs(next_path, exist_ok=True)
            download_files(file, next_path)
        else:
            # Download the file
            response = requests.get(file)
            file_path = os.path.join(download_path, file.split('/')[-1])
            with open(file_path, 'wb') as f:
                f.write(response.content)
            print(f'Downloaded {file_path}')

In [46]:
# base_url = input("Enter the base URL: ")
base_url = "https://plasmodb.org/common/downloads/Current_Release/Pfalciparum3D7"
# download_directory = input("Enter the path to the local directory for downloads: ")
download_directory = "/Users/dhruvparthasarathy/Documents/personal/harvard-lab-app/data"

# Create the base directory if it doesn't exist
os.makedirs(download_directory, exist_ok=True)

# Start the recursive download
download_files(base_url, download_directory)

href fasta/
href gaf/
href gff/
href txt/
href xml/
href data/
href PlasmoDB-68_Pfalciparum3D7_AnnotatedCDSs.fasta
href PlasmoDB-68_Pfalciparum3D7_AnnotatedProteins.fasta
href PlasmoDB-68_Pfalciparum3D7_AnnotatedTranscripts.fasta
href PlasmoDB-68_Pfalciparum3D7_Genome.fasta
Downloaded /Users/dhruvparthasarathy/Documents/personal/harvard-lab-app/data/fasta/data/PlasmoDB-68_Pfalciparum3D7_AnnotatedCDSs.fasta
Downloaded /Users/dhruvparthasarathy/Documents/personal/harvard-lab-app/data/fasta/data/PlasmoDB-68_Pfalciparum3D7_AnnotatedProteins.fasta
Downloaded /Users/dhruvparthasarathy/Documents/personal/harvard-lab-app/data/fasta/data/PlasmoDB-68_Pfalciparum3D7_AnnotatedTranscripts.fasta
Downloaded /Users/dhruvparthasarathy/Documents/personal/harvard-lab-app/data/fasta/data/PlasmoDB-68_Pfalciparum3D7_Genome.fasta
href PlasmoDB-68_Pfalciparum3D7_Curated_GO.gaf.gz
href PlasmoDB-68_Pfalciparum3D7_GO.gaf.gz
href PlasmoDB-CURRENT_Pfalciparum3D7_Curated_GO.gaf.gz
href PlasmoDB-CURRENT_Pfalciparum3